# Deploy models with the Azure Machine Learning service

The Azure Machine Learning service provides several ways you can deploy your trained model using the SDK. In this document, learn how to deploy your model as a web service in the Azure cloud, or to IoT edge devices.

## Register a trained model

The model registry is a way to store and organize your trained models in the Azure cloud. Models are registered in your Azure Machine Learning service workspace. The model can be trained using Azure Machine Learning, or another service. 

In [1]:
from azureml.core import Workspace

workspace_name = "aa-ml-aml-workspace"
subscription_id = "18522758-626e-4d88-92ac-dc9c7a5c26d4"
resource_group = "analytics.aml.experiments.workspaces"
ws = Workspace(workspace_name = workspace_name, subscription_id = subscription_id, resource_group=resource_group)

## Using Azure Portal

The easier way to do this is by going to the portal and registering a model using the UI. Go to portal.azure.com and open your experiment. Then you just open it by name

In [ ]:
model = ws.models['cats_vs_dogs']

### Using the output of a run

let's find the best run among all of them

In [18]:
from azureml.core import Run
exp = ws.experiments['azureml-cats-vs-dogs']
runs_accuracy = { run.id : run.get_metrics()['training_acc'] for run in exp.get_runs() }

best_run_id = max(runs_accuracy, key=lambda k : runs_accuracy[k])
best_run = Run(exp, best_run_id)

for f in best_run.get_file_names():
    print(f)

model = best_run.register_model(model_name='cats_vs_dogs', model_path='outputs/cats-vs-dogs.pth')

azureml-logs/55_batchai_execution.txt
azureml-logs/60_control_log.txt
azureml-logs/80_driver_log.txt
azureml-logs/azureml.log
outputs/cats-vs-dogs.pth


### If I have the model I want to register

In [20]:
from azureml.core.model import Model

model = Model.register(model_path = "/home/santiagxf/.fastai/data/dogscats/export.pkl",
                       model_name = "cats_vs_dogs",
                       tags = {"key": "0.1"},
                       description = "cats_vs_dogs",
                       workspace = ws)

Registering model cats_vs_dogs


# Create an score.py file

The execution script receives data submitted to a deployed image, and passes it to the model. It then takes the response returned by the model and returns that to the client. The script is specific to your model; it must understand the data that the model expects and returns. The script usually contains two functions that load and run the model:

init(): Typically this function loads the model into a global object. This function runs only once when the Docker container is started.

run(input_data): This function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization. You can also work with raw binary data. You can transform the data before sending to the model, or before returning to the client.

In [56]:
%%writefile score.py

import json
import torch
import os, base64
from fastai import *
from fastai.vision import *
from shutil import copyfile
from azureml.core.model import Model

def init():
    global learn
    
    model_file = Model.get_model_path("cats_vs_dogs")
    model_path = os.path.dirname(model_file)
    print(model_path)
    learn = load_learner(model_path)

def run(raw_data):
    base64_string = json.loads(raw_data)['data']
    base64_bytes = base64.b64decode(base64_string)
    with open(os.path.join(os.getcwd(),"score.jpg"), 'wb') as f:
        f.write(base64_bytes)
    
    # make prediction
    img = open_image(os.path.join(os.getcwd(),"score.jpg"))
    result = learn.predict(img)
    return json.dumps({'category':str(result[0]), 'confidence':result[2].data[1].item()})

Overwriting score.py


## Create and environment file

The environment file is the specific configuration you will use for your python environemnt you are using. Note that you may use the same conda_dependencies.yml you used for training or you may use a different one tailored for the scoring processes.

In [ ]:
!ls aml_config

In [60]:
%%writefile conda_dependencies.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
- dataclasses==0.6
- fastai==1.0.42
- pytorch==1.0.0
- torchvision==0.2.1
channels:
- pytorch
- fastai

Overwriting conda_dependencies.yml


## Create and register an image

Deployed models are packaged as an image. The image contains the dependencies needed to run the model.

For Azure Container Instance, Azure Kubernetes Service, and Azure IoT Edge deployments, the azureml.core.image.ContainerImage class is used to create an image configuration. The image configuration is then used to create a new Docker image. 

In [4]:
from azureml.core.image import ContainerImage

# Image configuration
image_config = ContainerImage.image_configuration(execution_script = "score.py", runtime = "python",
                                                 conda_file = "conda_dependencies.yml",
                                                 description = "Image classficiation service cats vs dogs",
                                                 tags = {"data": "cats-vs-dogs", "type": "classification"})

### Create the image



In [5]:
# Register the image from the image configuration
image = ContainerImage.create(name = "azureml-gpubenchmark-fastai", 
                              models = [model], #this is the model object
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output=True)

Creating image
Running.................................................................................................
SucceededImage creation operation finished for image azureml-gpubenchmark-fastai:8, operation "Succeeded"


## Desploy the image

When you get to deployment, the process is slightly different depending on the compute target that you deploy to. 

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                               memory_gb = 6, 
                                               tags = {"data": "cats-vs-dogs", "type": "classification"}, 
                                               description = 'Image classficiation service cats vs dogs')

## Deploy the service

In [ ]:
from azureml.core.webservice import Webservice

service_name = 'azureml-gpu-fastai-service'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)

In [64]:
service.wait_for_deployment(show_output = True)

Running...........................................
SucceededACI service creation operation finished, operation "Succeeded"


In [7]:
from azureml.core.webservice import Webservice
service = Webservice(name='azureml-gpu-fastai-service', workspace=ws)

In [8]:
service.get_logs()

'll}\n{"timestamp": "2019-04-05T12:04:53.784395Z", "message": "127.0.0.1 - - [05/Apr/2019:12:04:53 +0000] \\"POST /wanan.php HTTP/1.0\\" 404 233 \\"-\\" \\"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.105 Safari/537.36\\"", "host": "wk-caas-67a7ec66297344b7beaf3bed822f38e9-253041a86137628a89c9c9", "path": "/opt/miniconda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(asctime)s, %(levelname)s, %(message)s", "level": "INFO", "logger": "gunicorn.access", "stack_info": null}\n{"timestamp": "2019-04-05T12:04:54.039606Z", "message": "127.0.0.1 - - [05/Apr/2019:12:04:54 +0000] \\"POST /ssaa.php HTTP/1.0\\" 404 233 \\"-\\" \\"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.105 Safari/537.36\\"", "host": "wk-caas-67a7ec66297344b7beaf3bed822f38e9-253041a86137628a89c9c9", "path": "/opt/miniconda/lib/python3.6/site-packages/gunicorn/glogging.py", "tags": "%(module)s, %(as

## Test service

In [9]:
service = ws.services

AttributeError: 'Workspace' object has no attribute 'services'

In [12]:
import json, os
import base64

!wget -O test.jpg https://thenypost.files.wordpress.com/2018/05/180516-woman-mauled-by-angry-wiener-dogs-feature.jpg
    
with open(os.path.join(os.getcwd(),"test.jpg"), 'rb') as open_file:
    byte_content = open_file.read()

base64_bytes = base64.b64encode(byte_content)
base64_string = base64_bytes.decode('utf-8')

test_sample = json.dumps({'data': base64_string})
test_sample = bytes(test_sample,encoding = 'utf8')

prediction = service.run(input_data=test_sample)
print(prediction)

--2019-04-09 15:16:57--  https://www.artnews.com/wp-content/uploads/2015/09/chronology10.jpg
Resolving www.artnews.com (www.artnews.com)... 35.197.74.49
Connecting to www.artnews.com (www.artnews.com)|35.197.74.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35567 (35K) [image/jpeg]
Saving to: ‘test.jpg’

test.jpg            100%[===================>]  34.73K  --.-KB/s    in 0.07s   

2019-04-09 15:16:57 (484 KB/s) - ‘test.jpg’ saved [35567/35567]

{"category": "cats", "confidence": 0.26735371351242065}
